In [1]:
import cpa
import scanpy as sc
import pandas as pd
import anndata as ad
import numpy as np

[rank: 0] Global seed set to 0
2024-07-10 11:41:39,001	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2024-07-10 11:41:43,559	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [2]:
adata_train_path = "/lustre/groups/ml01/workspace/ot_perturbation/data/combosciplex/adata_train_300.h5ad"
adata_test_path = "/lustre/groups/ml01/workspace/ot_perturbation/data/combosciplex/adata_test_300.h5ad"
adata_ood_path = "/lustre/groups/ml01/workspace/ot_perturbation/data/combosciplex/adata_ood_300.h5ad"

In [3]:
adata_train = sc.read(adata_train_path)
adata_test = sc.read(adata_test_path)
adata_ood = sc.read(adata_ood_path)

In [4]:
adata_ood.obs["condition"].value_counts()

condition
Givinostat+SRT1720         2260
Cediranib+PCI-34051        2161
Panobinostat+PCI-34051     1814
Panobinostat+Crizotinib    1641
SRT2104+Alvespimycin        520
control                     500
Name: count, dtype: int64

In [5]:
try:
    data_path = 'combo_sciplex_prep_hvg_filtered.h5ad'
    adata = sc.read(data_path)
except:
    import gdown
    gdown.download('https://drive.google.com/uc?export=download&id=1RRV0_qYKGTvD3oCklKfoZQFYqKJy4l6t')
    data_path = 'combo_sciplex_prep_hvg_filtered.h5ad'
    adata = sc.read(data_path)

adata

adata.obs["index"] = adata.obs_names.values
adata.obs["index_adapted"] = adata.obs.apply(lambda x: x["index"][:-2], axis=1)
adata.obs.set_index("index_adapted", inplace=True)

adata_ood.obs["cov_drug_dose"] = adata.obs["cov_drug_dose"]
adata_ood.obs["condition_ID"] = adata.obs["condition_ID"]
adata_ood.obs["log_dose"] = adata.obs["log_dose"]
adata_ood.obs["smiles_rdkit"] = adata.obs["smiles_rdkit"]
adata_ood.obs["cell_type"] = adata.obs["cell_type"]

adata_test.obs["cov_drug_dose"] = adata.obs["cov_drug_dose"]
adata_test.obs["condition_ID"] = adata.obs["condition_ID"]
adata_test.obs["log_dose"] = adata.obs["log_dose"]
adata_test.obs["smiles_rdkit"] = adata.obs["smiles_rdkit"]
adata_test.obs["cell_type"] = adata.obs["cell_type"]

adata_train.obs["cov_drug_dose"] = adata.obs["cov_drug_dose"]
adata_train.obs["condition_ID"] = adata.obs["condition_ID"]
adata_train.obs["log_dose"] = adata.obs["log_dose"]
adata_train.obs["smiles_rdkit"] = adata.obs["smiles_rdkit"]
adata_train.obs["cell_type"] = adata.obs["cell_type"]

In [6]:
control_cells_gex = adata_ood[adata_ood.obs["condition"]=="control"].X.A
obs_names = list(adata_ood[adata_ood.obs["condition"]=="control"].obs_names)

all_data = []
obs_df = pd.DataFrame(columns=["condition_ID", "log_dose", "smiles_rdkit", "cell_type", "condition"])
indices = []

for condition in adata_ood.obs["condition"].cat.categories:
    if condition=="control":
        continue
    all_data.append(control_cells_gex)
    filtered_row = pd.Series(adata_ood[adata_ood.obs["condition"] == condition].obs[["condition_ID", "log_dose", "smiles_rdkit", "cell_type", "condition"]].iloc[0].values)
    filtered_row_df = pd.DataFrame([filtered_row.values], columns=["condition_ID", "log_dose", "smiles_rdkit", "cell_type", "condition"])
    duplicated_rows = pd.concat([filtered_row_df] * 500, ignore_index=True)
    obs_df = pd.concat([obs_df, duplicated_rows], ignore_index=True)

# Stack all data vertically to create a single array
all_data_array = np.vstack(all_data)


# Create the Anndata object
adata_ood_new = ad.AnnData(X=all_data_array, obs=obs_df)
adata_ood_new.var = adata_ood.var


control_cells_gex = adata_test[adata_test.obs["condition"]=="control"].X.A
obs_names = list(adata_test[adata_test.obs["condition"]=="control"].obs_names)

all_data = []
obs_df = pd.DataFrame(columns=["condition_ID", "log_dose", "smiles_rdkit", "cell_type", "condition"])
indices = []

for condition in adata_test.obs["condition"].cat.categories:
    if condition=="control":
        continue
    all_data.append(control_cells_gex)
    filtered_row = pd.Series(adata_test[adata_test.obs["condition"] == condition].obs[["condition_ID", "log_dose", "smiles_rdkit", "cell_type", "condition"]].iloc[0].values)
    filtered_row_df = pd.DataFrame([filtered_row.values], columns=["condition_ID", "log_dose", "smiles_rdkit", "cell_type", "condition"])
    duplicated_rows = pd.concat([filtered_row_df] * 500, ignore_index=True)
    obs_df = pd.concat([obs_df, duplicated_rows], ignore_index=True)

# Stack all data vertically to create a single array
all_data_array = np.vstack(all_data)


# Create the Anndata object
adata_test_new = ad.AnnData(X=all_data_array, obs=obs_df)
adata_test_new.var = adata_test.var

In [7]:
import anndata

adata_combined = anndata.concat((adata_train, adata_test_new, adata_ood_new), label="split", keys=["train", "test", "ood"], join="inner")

In [8]:
frac_valid = adata[adata.obs["split_1ct_MEC"] == "valid"].n_obs / (
    adata[adata.obs["split_1ct_MEC"] == "valid"].n_obs + adata[adata.obs["split_1ct_MEC"] == "train"].n_obs
)

In [9]:
import numpy as np


def create_split(x):
    if x["split"] != "train":
        return x["split"]
    is_train = np.random.choice(2, p=[frac_valid, 1 - frac_valid])
    if is_train:
        return "train_train"
    return "train_valid"


adata_combined.obs["split_1ct_MEC"] = adata_combined.obs.apply(create_split, axis=1)

In [11]:
cpa.CPA.setup_anndata(adata_combined,
                      perturbation_key='condition_ID',
                      dosage_key='log_dose',
                      control_group='CHEMBL504',
                      batch_key=None,
                      smiles_key='smiles_rdkit',
                      is_count_data=False,
                      categorical_covariate_keys=['cell_type'],
                      deg_uns_cat_key='cov_drug_dose',
                      max_comb_len=2,
                     )

100%|██████████| 67382/67382 [00:00<00:00, 1136291.09it/s]
An NVIDIA GPU may be present on this machine, but a CUDA-enabled jaxlib is not installed. Falling back to cpu.


INFO     Generating sequential column names                                                                        
INFO     Generating sequential column names                                                                        


In [12]:
ae_hparams = {'n_latent': 64,
 'recon_loss': 'gauss',
 'doser_type': 'linear',
 'n_hidden_encoder': 256,
 'n_layers_encoder': 3,
 'n_hidden_decoder': 512,
 'n_layers_decoder': 2,
 'use_batch_norm_encoder': True,
 'use_layer_norm_encoder': False,
 'use_batch_norm_decoder': True,
 'use_layer_norm_decoder': False,
 'dropout_rate_encoder': 0.25,
 'dropout_rate_decoder': 0.25,
 'variational': False,
 'seed': 6478}

trainer_params = {'n_epochs_kl_warmup': None,
 'n_epochs_pretrain_ae': 50,
 'n_epochs_adv_warmup': 100,
 'n_epochs_mixup_warmup': 10,
 'mixup_alpha': 0.1,
 'adv_steps': None,
 'n_hidden_adv': 128,
 'n_layers_adv': 3,
 'use_batch_norm_adv': False,
 'use_layer_norm_adv': False,
 'dropout_rate_adv': 0.2,
 'reg_adv': 10.0,
 'pen_adv': 0.1,
 'lr': 0.0003,
 'wd': 4e-07,
 'adv_lr': 0.0003,
 'adv_wd': 4e-07,
 'adv_loss': 'cce',
 'doser_lr': 0.0003,
 'doser_wd': 4e-07,
 'do_clip_grad': False,
 'gradient_clip_value': 1.0,
 'step_size_lr': 10}

In [13]:
model = cpa.CPA(adata=adata_combined,
                split_key='split_1ct_MEC',
                train_split='train_train',
                valid_split='train_valid',
                test_split='ood',
                use_rdkit_embeddings=True,
                **ae_hparams,
               )

(18, 2048)


[rank: 0] Global seed set to 6478


In [15]:
model.train(max_epochs=200,
            use_gpu=True,
            batch_size=512,
            plan_kwargs=trainer_params,
            early_stopping_patience=10,
            check_val_every_n_epoch=5,
            save_path="/lustre/groups/ml01/workspace/ot_perturbation/models/chemcpa/combosciplex",
           )

100%|██████████| 32/32 [00:00<00:00, 172.58it/s]
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA A100 80GB PCIe') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Epoch 5/200:   2%|▏         | 4/200 [00:37<23:07,  7.08s/it, v_num=1, recon=-2.44e+3, r2_mean=0.909, adv_loss=2.43, acc_pert=0.156]


Epoch 00004: cpa_metric reached. Module best state updated.


Epoch 10/200:   4%|▍         | 9/200 [01:06<17:33,  5.51s/it, v_num=1, recon=-2.85e+3, r2_mean=0.918, adv_loss=2.43, acc_pert=0.166, val_recon=-2.65e+3, disnt_basal=0.108, disnt_after=0.29, val_r2_mean=0.902, val_KL=nan]
disnt_basal = 0.09980377265869328
disnt_after = 0.20788828756201239
val_r2_mean = 0.9086304697555694
val_r2_var = 0.7044784349886997
Epoch 20/200:  10%|▉         | 19/200 [01:53<13:48,  4.58s/it, v_num=1, recon=-3.11e+3, r2_mean=0.927, adv_loss=2.46, acc_pert=0.166, val_recon=-2.97e+3, disnt_basal=0.0945, disnt_after=0.181, val_r2_mean=0.917, val_KL=nan]
disnt_basal = 0.08802382931019331
disnt_after = 0.1633828633051775
val_r2_mean = 0.9154551795837453
val_r2_var = 0.7268406598254887
Epoch 30/200:  14%|█▍        | 29/200 [02:41<13:10,  4.62s/it, v_num=1, recon=-3.28e+3, r2_mean=0.928, adv_loss=2.47, acc_pert=0.163, val_recon=-3.13e+3, disnt_basal=0.0817, disnt_after=0.148, val_r2_mean=0.921, val_KL=nan]
disnt_basal = 0.07465233165843886
disnt_after = 0.1417650180657335

In [16]:
model.predict(adata_combined)

100%|██████████| 2106/2106 [00:09<00:00, 223.32it/s]


In [18]:
adata_combined.write(
    "/lustre/groups/ml01/workspace/ot_perturbation/models/chemcpa/combosciplex/adata_with_predictions.h5ad"
)